In [1]:
import pandas as pd
import seaborn as sns
import numpy as np

In [2]:
from sklearn.datasets import load_iris

In [3]:
def round_to_3(x):
    
    return round(x, 3)

## Decision tree

In [4]:
class Node:
    def __init__(self, 
                 factor_ind, 
                 threshold, 
                 left_node, 
                 right_node):
    """
    Constructor for the tree node object.
    
    Arguments:
        factor_ind: Index of the factor used for the question at the node.
        threshold: Threshold value for the factor at the node.
        left_node: Node containing the left subtree of the current node.
        right_node: Node containing the right subtree of the current node.
    """

        
        self.factor_ind = factor_ind
        self.threshold = threshold
        
        self.left_node = left_node
        self.right_node = right_node
        
    def is_leaf(self):
    
    """
    Returns False, indicating that the object is not a leaf.
    """
        
        return False
        
class Leaf:
    def __init__(self, 
                 predicted_value):
        """
        Constructor for the tree leaf object.
        
        Arguments:
            predicted_value: Value that is the prediction at this leaf.
        """
        
        self.predicted_value = predicted_value
        
    def is_leaf(self):
        """
        Returns True, indicating that the object is a leaf.
        """
        
        return True

IndentationError: expected an indented block after function definition on line 2 (2746319842.py, line 7)

In [5]:
def print_tree(cur_node, s=''):
    """
    Prints the tree structure to the screen.
    
    Arguments:
        cur_node: Root node of the tree.
        s: Optional parameter for indentation.
    
    Returns:
        Prints the tree in the format:
        Node 0 (root node)
            Node 1 (left child)
                ... until leaves
            Node 2 (right child)
                ... until leaves
    """
    
    if cur_node.is_leaf():
        print(s + f'Лист(предсказание={cur_node.predicted_value})')
        return
    
    print(s + f'Вершина(фактор={cur_node.factor_ind}, порог={cur_node.threshold})')
    
    print_tree(cur_node.left_node, s + '    ')
    print_tree(cur_node.right_node, s + '    ')

In [9]:
tree = \
    Node(0, 5.5,
         Node(1, 2.9,
            Leaf(1.0),
            Leaf(0.0)),
         Node(0, 6.3,
            Leaf(1.0),
            Leaf(2.0)))

In [10]:
print_tree(tree)

Вершина(фактор=0, порог=5.5)
    Вершина(фактор=1, порог=2.9)
        Лист(предсказание=1.0)
        Лист(предсказание=0.0)
    Вершина(фактор=0, порог=6.3)
        Лист(предсказание=1.0)
        Лист(предсказание=2.0)


In [6]:
def decision_tree_predict_solution(cur_node, x):
    """
    Predicts the value for object x using the given decision tree.
    
    Arguments:
        cur_node: Root node of the tree used for prediction.
        x: Object for which the prediction is made.
    
    Returns:
        Predicted value for object x.
        If the leaves contain classes, the class is returned.
        If the leaves contain numerical values, the number is returned.
    """
    
    if cur_node.is_leaf():
        return cur_node.predicted_value

    if x[cur_node.factor_ind] >= cur_node.threshold:
        result = decision_tree_predict_solution(cur_node.right_node, x)
    else:
        result = decision_tree_predict_solution(cur_node.left_node, x)

    return result

In [7]:
def gini_index_solution(X):
    """
    Calculates the impurity of the data using the Gini criterion.
    
    Arguments:
        X: Set of objects represented as an n x m matrix.
           Each row corresponds to an object.
           Each object is described by (m - 1) factors and a class.
           The class is represented by a number.
           The index of the class in the object row is (m - 1), assuming 0-based indexing.
    
    Returns:
        Calculated impurity of the data, rounded to 3 decimal places.
    """

    res = dict()
    gini = 0
    
    for obj in X:
        res[obj[-1]] = res.get(obj[-1], 0) + 1
        
    for k, v in res.items():
        pi = v / len(X)
        gini += pi * (1 - pi)
        
    return round_to_3(gini)

In [108]:
x = {2: 20, 3: 10}
g = 0

for k, v in x.items():
        pi = v / 50
        g += pi * (1 - pi)

In [109]:
best = 0, 0
bests = []
for k, v in x.items():
    if v > best[1]:
        bests = []
        best = k, v
        bests.append(best)
    elif v == best[1]:
        bests.append((k, v))
print(sorted(bests)[0][0])

2


In [53]:
a = 0.62
b = 0.32
c = 0.64
n = 100
n1 = 50

In [54]:
H = a - (n1 / n) * b - (n1 / n) * c

In [55]:
H

0.13999999999999996

In [1]:
def data_split_solution(X, factor_ind, factor_value):
    
    X_l = X[X[:, factor_ind] < factor_value]
    X_r = X[X[:, factor_ind] >= factor_value]

    return X_l, X_r

In [2]:
def optimal_split_parameters_grid_search_solution(X):

    ind_best = 0
    val_best = 0
    Qmax = -1
    
    for i in range(len(X[0]) - 1):
    
        for f in (X[:, i]):
            
            X_l, X_r = data_split_solution(X, i, f)
            H = gini_index_solution(X)
            H_l = gini_index_solution(X_l)
            H_r = gini_index_solution(X_r)
            Q = H - (len(X_l) / len(X)) * H_l - (len(X_r) / len(X)) * H_r
            
            if Q > Qmax:
                Qmax = Q
                ind_best = i
                val_best = f
                
    return ind_best, val_best

In [81]:
X = np.array([
        [1, 2, -1],
        [2, 3, 1],
        [5, 4, 1],
        [2, 5, 1]
    ])

In [82]:
optimal_split_parameters_grid_search_solution(X)

(0, 2)

In [8]:
def build_node_solution(X, cur_depth=1, max_depth=None):
    """
    Builds a decision tree.
    
    Arguments:
        X: n x m matrix of objects. Rows are objects, columns (m-1) are factors, last column is the class.
        cur_depth: Current depth, default is 1.
        max_depth: Maximum depth, default is None.
    
    Returns:
        Decision tree built from X, either a node with subtrees or a leaf with a prediction.
"""
    
    res = {}
    
    for obj in X:
        res[obj[-1]] = res.get(obj[-1], 0) + 1
        
    if len(res) == 1:
        return Leaf(list(res.keys())[0])

    if max_depth is not None:
        
        if cur_depth > max_depth:
            best = 0, 0
            bests = []
            
            for k, v in res.items():
                
                if v > best[1]:
                    bests = []
                    best = k, v
                    bests.append(best)
                    
                elif v == best[1]:
                    bests.append((k, v))
                    
            return Leaf(sorted(bests)[0][0])

    factor, value = optimal_split_parameters_grid_search_solution(X)
    X_l, X_r = data_split_solution(X, factor, value)

    left_node = build_node_solution(X_l, cur_depth + 1, max_depth=max_depth)
    right_node = build_node_solution(X_r, cur_depth + 1, max_depth=max_depth)

    return Node(factor, value, left_node, right_node)

In [156]:
X = np.array([
        [-1, 2],
        [1, 3],
        [2, 2],
        [3, 3],
        [4, 2]
    ])
max_depth = 1
    
res_example_3 = \
    Node(0, 1.0,
        Leaf(2.0),
        Leaf(2.0))

In [157]:
print_tree(build_node_solution(X, cur_depth=1, max_depth=max_depth))

[[-1  2]
 [ 1  3]
 [ 2  2]
 [ 3  3]
 [ 4  2]] {2: 3, 3: 2}
[[-1  2]] {2: 1}
!!! 2
[[1 3]
 [2 2]
 [3 3]
 [4 2]] {3: 2, 2: 2}
! [(3, 2), (2, 2)]
Вершина(фактор=0, порог=1)
    Лист(предсказание=2)
    Лист(предсказание=2)


In [158]:
iris = load_iris()
    
X = iris.data
y = iris.target.reshape((len(X), 1))
    
X_example_1 = np.hstack([X[:, :2], y])
max_depth_example_1 = 2
    
res_example_1 = \
        Node(0, 5.5,
             Node(1, 2.9,
                 Leaf(1.0),
                 Leaf(0.0)),
             Node(0, 6.3,
                 Leaf(1.0),
                 Leaf(2.0)))
    
print_tree(build_node_solution(X_example_1, max_depth=max_depth_example_1))

[[5.1 3.5 0. ]
 [4.9 3.  0. ]
 [4.7 3.2 0. ]
 [4.6 3.1 0. ]
 [5.  3.6 0. ]
 [5.4 3.9 0. ]
 [4.6 3.4 0. ]
 [5.  3.4 0. ]
 [4.4 2.9 0. ]
 [4.9 3.1 0. ]
 [5.4 3.7 0. ]
 [4.8 3.4 0. ]
 [4.8 3.  0. ]
 [4.3 3.  0. ]
 [5.8 4.  0. ]
 [5.7 4.4 0. ]
 [5.4 3.9 0. ]
 [5.1 3.5 0. ]
 [5.7 3.8 0. ]
 [5.1 3.8 0. ]
 [5.4 3.4 0. ]
 [5.1 3.7 0. ]
 [4.6 3.6 0. ]
 [5.1 3.3 0. ]
 [4.8 3.4 0. ]
 [5.  3.  0. ]
 [5.  3.4 0. ]
 [5.2 3.5 0. ]
 [5.2 3.4 0. ]
 [4.7 3.2 0. ]
 [4.8 3.1 0. ]
 [5.4 3.4 0. ]
 [5.2 4.1 0. ]
 [5.5 4.2 0. ]
 [4.9 3.1 0. ]
 [5.  3.2 0. ]
 [5.5 3.5 0. ]
 [4.9 3.6 0. ]
 [4.4 3.  0. ]
 [5.1 3.4 0. ]
 [5.  3.5 0. ]
 [4.5 2.3 0. ]
 [4.4 3.2 0. ]
 [5.  3.5 0. ]
 [5.1 3.8 0. ]
 [4.8 3.  0. ]
 [5.1 3.8 0. ]
 [4.6 3.2 0. ]
 [5.3 3.7 0. ]
 [5.  3.3 0. ]
 [7.  3.2 1. ]
 [6.4 3.2 1. ]
 [6.9 3.1 1. ]
 [5.5 2.3 1. ]
 [6.5 2.8 1. ]
 [5.7 2.8 1. ]
 [6.3 3.3 1. ]
 [4.9 2.4 1. ]
 [6.6 2.9 1. ]
 [5.2 2.7 1. ]
 [5.  2.  1. ]
 [5.9 3.  1. ]
 [6.  2.2 1. ]
 [6.1 2.9 1. ]
 [5.6 2.9 1. ]
 [6.7 3.1 1. ]
 [5.6 3.  

In [159]:
X_example_2 = np.hstack([X, y])
max_depth_example_2 = None
    
res_example_2 = \
        Node(2, 3.0,
             Leaf(0.0),
             Node(3, 1.8,
                 Node(2, 5.0,
                     Node(3, 1.7,
                         Leaf(1.0),
                         Leaf(2.0)),
                     Node(3, 1.6,
                         Leaf(2.0),
                         Node(0, 7.2,
                             Leaf(1.0),
                             Leaf(2.0)))),
                 Node(2, 4.9,
                     Node(0, 6.0,
                         Leaf(1.0),
                         Leaf(2.0)),
                     Leaf(2.0))))
    
print_tree(build_node_solution(X_example_2, max_depth=max_depth_example_2))

[[5.1 3.5 1.4 0.2 0. ]
 [4.9 3.  1.4 0.2 0. ]
 [4.7 3.2 1.3 0.2 0. ]
 [4.6 3.1 1.5 0.2 0. ]
 [5.  3.6 1.4 0.2 0. ]
 [5.4 3.9 1.7 0.4 0. ]
 [4.6 3.4 1.4 0.3 0. ]
 [5.  3.4 1.5 0.2 0. ]
 [4.4 2.9 1.4 0.2 0. ]
 [4.9 3.1 1.5 0.1 0. ]
 [5.4 3.7 1.5 0.2 0. ]
 [4.8 3.4 1.6 0.2 0. ]
 [4.8 3.  1.4 0.1 0. ]
 [4.3 3.  1.1 0.1 0. ]
 [5.8 4.  1.2 0.2 0. ]
 [5.7 4.4 1.5 0.4 0. ]
 [5.4 3.9 1.3 0.4 0. ]
 [5.1 3.5 1.4 0.3 0. ]
 [5.7 3.8 1.7 0.3 0. ]
 [5.1 3.8 1.5 0.3 0. ]
 [5.4 3.4 1.7 0.2 0. ]
 [5.1 3.7 1.5 0.4 0. ]
 [4.6 3.6 1.  0.2 0. ]
 [5.1 3.3 1.7 0.5 0. ]
 [4.8 3.4 1.9 0.2 0. ]
 [5.  3.  1.6 0.2 0. ]
 [5.  3.4 1.6 0.4 0. ]
 [5.2 3.5 1.5 0.2 0. ]
 [5.2 3.4 1.4 0.2 0. ]
 [4.7 3.2 1.6 0.2 0. ]
 [4.8 3.1 1.6 0.2 0. ]
 [5.4 3.4 1.5 0.4 0. ]
 [5.2 4.1 1.5 0.1 0. ]
 [5.5 4.2 1.4 0.2 0. ]
 [4.9 3.1 1.5 0.2 0. ]
 [5.  3.2 1.2 0.2 0. ]
 [5.5 3.5 1.3 0.2 0. ]
 [4.9 3.6 1.4 0.1 0. ]
 [4.4 3.  1.3 0.2 0. ]
 [5.1 3.4 1.5 0.2 0. ]
 [5.  3.5 1.3 0.3 0. ]
 [4.5 2.3 1.3 0.3 0. ]
 [4.4 3.2 1.3 0.2 0. ]
 [5.  3.5 1

In [9]:
def compare_trees(tree_1, tree_2):
    """
    Checks if two trees are equal.
    
    Arguments:
        tree_1: First tree.
        tree_2: Second tree.
    
    Returns:
        True if the trees are equal.
        False otherwise.
    """
    
    if tree_1.is_leaf() != tree_2.is_leaf():
        return False
    
    if tree_1.is_leaf() and tree_2.is_leaf():
        return int(tree_1.predicted_value) == int(tree_2.predicted_value)
    
    return tree_1.factor_ind == tree_2.factor_ind and \
        tree_1.threshold == tree_2.threshold and \
        compare_trees(tree_1.left_node, tree_2.left_node) and \
        compare_trees(tree_1.right_node, tree_2.right_node)

def build_node_test():
    iris = load_iris()
    
    X = iris.data
    y = iris.target.reshape((len(X), 1))
    
    X_example_1 = np.hstack([X[:, :2], y])
    max_depth_example_1 = 2
    
    res_example_1 = \
        Node(0, 5.5,
             Node(1, 2.9,
                 Leaf(1.0),
                 Leaf(0.0)),
             Node(0, 6.3,
                 Leaf(1.0),
                 Leaf(2.0)))
    
    assert compare_trees(build_node_solution(X_example_1, max_depth=max_depth_example_1), res_example_1)
    
    X_example_2 = np.hstack([X, y])
    max_depth_example_2 = None
    
    res_example_2 = \
        Node(2, 3.0,
             Leaf(0.0),
             Node(3, 1.8,
                 Node(2, 5.0,
                     Node(3, 1.7,
                         Leaf(1.0),
                         Leaf(2.0)),
                     Node(3, 1.6,
                         Leaf(2.0),
                         Node(0, 7.2,
                             Leaf(1.0),
                             Leaf(2.0)))),
                 Node(2, 4.9,
                     Node(0, 6.0,
                         Leaf(1.0),
                         Leaf(2.0)),
                     Leaf(2.0))))
    
    assert compare_trees(build_node_solution(X_example_2, max_depth=max_depth_example_2), res_example_2)
    
    X_example_3 = np.array([
        [-1, 2],
        [1, 3],
        [2, 2],
        [3, 3],
        [4, 2]
    ])
    max_depth_example_3 = 1
    
    res_example_3 = \
        Node(0, 1.0,
             Leaf(2.0),
             Leaf(2.0))
    
    assert compare_trees(build_node_solution(X_example_3, max_depth=max_depth_example_3), res_example_3)
    
    print('Suscess!')

In [165]:
build_node_test()

Тест прошёл успешно!
